# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine

# NLP
import nltk
nltk.download(['punkt', 'wordnet'])
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# ML
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)

X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # remove punctations
    text =  ''.join([c for c in text if c not in punctuation])
    
    #tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for token in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def evalute_model(pipeline, X_test, y_test, category_names):

    # predict on test data
    y_pred = pipeline.predict(X_test)
    
    performances = []
    # print metries
    for i, col in enumerate(category_names):
        precision, recall, f_score, support = score(y_test.iloc[:,i], y_pred[:,i], average='micro')
        performances.append([precision, recall, f_score])
        
    performances = pd.DataFrame(performances, columns=['precesion', 'recall', 'f1-score'],
                                index = category_names) 
    print(performances)

evalute_model(pipeline, X_test, y_test, category_names)

                        precesion    recall  f1-score
related                  0.783644  0.783644  0.783644
request                  0.879921  0.879921  0.879921
offer                    0.996033  0.996033  0.996033
aid_related              0.727952  0.727952  0.727952
medical_help             0.917608  0.917608  0.917608
medical_products         0.952395  0.952395  0.952395
search_and_rescue        0.971773  0.971773  0.971773
security                 0.980317  0.980317  0.980317
military                 0.967348  0.967348  0.967348
child_alone              1.000000  1.000000  1.000000
water                    0.950259  0.950259  0.950259
food                     0.911962  0.911962  0.911962
shelter                  0.927983  0.927983  0.927983
clothing                 0.985047  0.985047  0.985047
money                    0.975435  0.975435  0.975435
missing_people           0.989319  0.989319  0.989319
refugees                 0.967653  0.967653  0.967653
death                    0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__max_features':['sqrt'],
    'clf__estimator__n_estimators':[10]
}

cv = GridSearchCV(estimator=pipeline, param_grid = parameters, cv = 3)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
evalute_model(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_improved = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())) 
])

# train classifier
pipeline_improved.fit(X_train, y_train)
evalute_model(pipeline_improved, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))
pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.